<a href="https://colab.research.google.com/github/samsoe/mpg_notebooks/blob/master/abundance_order_habitat_wrangle.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

Documentation: [Readme bird point count monitoring](https://docs.google.com/document/d/1PRryJzGOUtfr-fKXzb3tbr48xiaTuAMVk18XFXlvxcM/edit?usp=sharing)

# Tools

In [0]:
library(tidyverse)

In [0]:
install.packages("bigrquery")
library(bigrquery)

# Source

## BigQuery

### Database Connection

In [0]:
# BigQuery API Key
bq_auth(path = "/content/mpg-data-warehouse-34434e1a9914.json")

In [0]:
Sys.setenv(BIGQUERY_TEST_PROJECT = "mpg-data-warehouse")

In [0]:
billing <- bq_test_project()

In [0]:
con_bird_point_counts <- dbConnect(
  bigrquery::bigquery(),
  project = "mpg-data-warehouse",
  dataset = "bird_point_counts",
  billing = billing
)

In [0]:
con_location_position_classification <- dbConnect(
    bigrquery::bigquery(),
  project = "mpg-data-warehouse",
  dataset = "grid_point_summaries",
  billing = billing
)

In [11]:
dbListTables(con_bird_point_counts)

[1] "bird_surveyInfo_records_function" "bird_survey_noise"               
[3] "bird_survey_sky"                  "bird_survey_wind"

In [15]:
dbListTables(con_location_position_classification)

[1] "location"                         "location_position_classification"
[3] "point_location_position"

### Query Database

#### bird_point_counts

In [0]:
bird_sql <- "SELECT *
             FROM `mpg-data-warehouse.bird_point_counts.bird_surveyInfo_records_function`"

In [0]:
# result bird table
bq_bird <- bq_project_query(billing, bird_sql)

In [0]:
tb_bird = bq_table_download(bq_bird)

In [25]:
class(tb_bird)

[1] "tbl_df"     "tbl"        "data.frame"

In [0]:
df_bird <- as.data.frame(tb_bird)

In [33]:
str(df_bird)

'data.frame':	117810 obs. of  31 variables:
 $ survey_ID                         : int  5195 8598 8599 4294 4295 4296 4297 4302 10 22 ...
 $ survey_year                       : int  2013 2015 2015 2013 2013 2013 2013 2013 2010 2010 ...
 $ survey_date                       : Date, format: "2013-05-29" "2015-06-09" ...
 $ survey_visit_calc                 : int  1 1 1 1 1 1 1 1 1 1 ...
 $ survey_time_start_MDT             : 'hms' num  09:38:00 06:43:00 07:02:00 06:13:00 ...
  ..- attr(*, "units")= chr "secs"
 $ survey_time_end_MDT               : 'hms' num  09:48:00 06:53:00 07:12:00 06:23:00 ...
  ..- attr(*, "units")= chr "secs"
 $ survey_grid_point                 : int  163 482 480 193 194 195 181 166 477 484 ...
 $ survey_observer                   : chr  "Katharine Stone" "Eric Rasmussen" "Eric Rasmussen" "Eric Rasmussen" ...
 $ survey_wind_code                  : int  2 1 0 1 2 1 1 2 1 2 ...
 $ survey_sky_code                   : int  1 0 0 0 0 0 0 1 2 2 ...
 $ survey_noise_code  

#### location_position_classification

In [0]:
location_sql <- "SELECT *
                 FROM `mpg-data-warehouse.grid_point_summaries.location_position_classification`"

Leave the following error for now.  To move forward in the present output csv from table output csv from `mpg-data-warehouse.grid_point_summaries.location_position_classification`

In [29]:
bq_location <- bq_project_query(billing, location_sql)

ERROR: ignored

## CSV

### location_position_classification

In [30]:
# MPG Ranch Datasets > Bird point count monitoring > Data - Processed > location_position_classification.csv
src_location = "https://drive.google.com/uc?id=1z1nFdsk0vZXhw78rB3ijrFcLX2mF4VBQ"
df_location <- read_csv(src_location)

Parsed with column specification:
cols(
  grid_point = col_double(),
  lat = col_double(),
  long = col_double(),
  aspect_mean_deg = col_double(),
  elevation_mean_m = col_double(),
  slope_mean_deg = col_double(),
  cover_type_2016_gridVeg = col_character(),
  biomass_habitat_type = col_character(),
  type1_biome = col_character(),
  type2_vegetation_community = col_character(),
  type3_vegetation_indicators = col_character(),
  type4_indicators_history = col_character()
)



# Structure

## bird_point_counts

In [0]:
bird_abundance <- df_bird %>%
  group_by(survey_year, survey_visit_calc, survey_grid_point, function_taxa_order) %>%
  filter(records_detect_distance_min_meters <= 100,
         records_interval %in% c(1, 2)) %>%
  summarize(bird_abundance = sum(records_abundance)) %>%
  group_by(survey_year, survey_grid_point, function_taxa_order) %>%
  summarize(bird_abundance_avg = mean(bird_abundance))

# Join

In [51]:
names(bird_abundance)

[1] "survey_year"         "survey_grid_point"   "function_taxa_order"
[4] "bird_abundance_avg"

In [52]:
names(df_location)

[1] "grid_point"                  "lat"                        
 [3] "long"                        "aspect_mean_deg"            
 [5] "elevation_mean_m"            "slope_mean_deg"             
 [7] "cover_type_2016_gridVeg"     "biomass_habitat_type"       
 [9] "type1_biome"                 "type2_vegetation_community" 
[11] "type3_vegetation_indicators" "type4_indicators_history"

In [56]:
bird_abundance %>% 
  left_join(df_location, by = c("survey_grid_point" = "grid_point")) %>%
  head()

survey_year,survey_grid_point,function_taxa_order,bird_abundance_avg,lat,long,aspect_mean_deg,elevation_mean_m,slope_mean_deg,cover_type_2016_gridVeg,biomass_habitat_type,type1_biome,type2_vegetation_community,type3_vegetation_indicators,type4_indicators_history
<int>,<dbl>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<chr>,<chr>,<chr>,<chr>,<chr>,<chr>
2010,1,Passeriformes,1.0,46.73193,-114.0017,334.705,1395.64,28.44230,woodland/forest,NA,forest,upland,mixed canopy conifer,mixed canopy conifer
2010,2,Passeriformes,1.0,46.72972,-114.0010,45.303,1456.09,12.22630,non-irrigated grasslands,Range,rangeland,grassland,uncultivated grassland native or degraded,uncultivated grassland native or degraded
2010,3,Passeriformes,1.0,46.72443,-114.0227,221.334,1126.90,4.25130,shrubland,Range,rangeland,grassland,uncultivated grassland native or degraded,uncultivated grassland native or degraded
2010,4,Passeriformes,1.0,46.72487,-114.0195,290.489,1166.33,2.68361,shrubland,Range,rangeland,grassland,uncultivated grassland native or degraded,uncultivated grassland native or degraded
2010,7,Accipitriformes,1.0,46.72707,-114.0036,302.818,1387.80,26.17570,non-irrigated grasslands,Range,rangeland,grassland,uncultivated grassland native or degraded,uncultivated grassland native or degraded
2010,7,Passeriformes,1.5,46.72707,-114.0036,302.818,1387.80,26.17570,non-irrigated grasslands,Range,rangeland,grassland,uncultivated grassland native or degraded,uncultivated grassland native or degraded
